In [1]:
import pandas as pd
import numpy as np
import _pickle
import datetime

In [2]:
# load and reorganize data.
df = pd.read_csv("csvs/zansa.csv", names=["ts", "lat", "lng", "text"])
df = df.drop([0,1,2,3,4,1118,1281]) # 最初のテストデータの消去と外れ値の消去
# df.index = np.arange(len(df.index))
df2 = pd.DataFrame(np.array(df), columns=df.columns) # ダミー
df2['ts'] = pd.to_datetime(df2['ts']) # ts列をstrからtimestampへ
df2['text'] = df2['text'].astype('str') # text列を念のため，すべてstrに変換

In [3]:
# テスト抜きのdfを作成
test_index = list(df2[df2['text'].str.contains('test|テス|てすと|試し|エクス|ファイ')].index)
df3 = df2.drop(test_index)
print(len(df3))

1533


In [4]:
with open('pkls/m3_list.pkl', 'rb') as f:
    mesh_list = _pickle.load(f)
kanagawa_mesh = pd.read_csv('csvs/14.csv')

### 藤沢市の地域メッシュコードのリストを作成

In [5]:
len(kanagawa_mesh)

3497

In [6]:
fujisawa_index = []
for index, row in kanagawa_mesh.iterrows():
    if '藤沢市' in str(row['市区町村名']):
        fujisawa_index.append(index)
fujisawa_mesh = kanagawa_mesh[1687:1789]


In [7]:
fujisawa_mesh.head(5)

,都道府県市区町村コード,市区町村名,基準メッシュコード
1687,14205,藤沢市,52397357
1688,14205,藤沢市,52397358
1689,14205,藤沢市,52397359
1690,14205,藤沢市,52397367
1691,14205,藤沢市,52397368


In [8]:
fujisawa_mesh_list = list(fujisawa_mesh['基準メッシュコード'])
print(len(fujisawa_mesh_list))
rm_list = [53390324,53390334,53390352,53390351,53390361,53390381,53390372,53391303,53391304,53391319,53391308,53390359,53390349,53390339,53390411,53390401,53397490,53397369,53397357,53397367,53397376]
print(len(rm_list))
for i in rm_list:
    try:
        fujisawa_mesh_list.remove(i)
    except:
        print(i)
print(len(fujisawa_mesh_list))
fm_array = np.array(fujisawa_mesh_list)
fm1_list = list(fm_array * 10 + 1)
fm2_list = list(fm_array * 10 + 2)
fm3_list = list(fm_array * 10 + 3)
fm4_list = list(fm_array * 10 + 4)
fm_list = fm1_list + fm2_list + fm3_list + fm4_list
print(len(fm_list))

102
21
53390351
53391319
53397490
53397369
53397357
53397367
53397376
88
352


In [9]:
with open('pkls/fm_list.pkl', 'wb') as f:
    _pickle.dump(fm_list, f)

### 3次地域メッシュコードの追加

In [10]:
import re  # 正規表現によるパターンマッチ

def meshcode_to_latlng(code, loc = 'C'):

	if type(code) != int:  # 整数型に（文字列が渡された場合に対応）
		try:
			code = int(code)
		except:  # 形式チェック
			print("Error in  mesh_lib.meshocode_to_latlong()  Not a valid code handed.")
			raise

	code = str(code) # あらためて文字型に。

	code12 = ''
	code34 = ''
	code5  = ''
	code6  = ''
	code7  = ''
	code8  = ''

	loc = loc.upper()

	if re.match("^C|(NE)|(NW)|(SE)|(SW)$", loc) == None:
		raise Exception ("Invalid option to meshcode_to_latlong()")

	match_1_result = re.match("\d{4}", code)

	if re.match("\d{4}", code):  # 最初の４文字が数字

		code12 = code[0:2]
		code34 = code[2:4]
		lat_width  = 2.0 / 3.0  # grid cell の緯度方向の間隔
		long_width = 1.0        # grid cell の経度方向の間隔

	else:
		return (None)           # メッシュコードとして無効

	if re.match("\d{6}", code):   # 少なくとも最初の６文字は数字

		code5 = code[4:5]
		code6 = code[5:6]
		lat_width  /= 8.0;
		long_width /= 8.0;

	if re.match("\d{8}", code):  # 最初の８文字は数字

		code7 = code[6:7]
		code8 = code[7:8]
		lat_width  /= 10.0;
		long_width /= 10.0;

    # 以下、南西コーナーの座標を求める。

	lat  = float(code12) * 2 / 3          #  一次メッシュ
	long = float(code34) + 100

	if (code5 != '') & (code6 != ''):     # 二次メッシュ or 三次メッシュ
		lat  += (float(code5) * 2 / 3) / 8
		long += float(code6) / 8 

	if (code7 != '') & (code8 != ''):     # 三次メッシュ
		lat  += float(code7) * 2 / 3 / 8 / 10
		long += float(code8) / 8 / 10 

	# ここまでで南東端の緯度・経度が得られた

	# 中央の座標なら、一区画の幅（経度幅）・高さ（緯度幅）の半分を足す。
	if loc == 'C':
		lat  += lat_width  / 2 
		long += long_width / 2

	# 北端の座標なら、一区画の高さ（緯度幅）を足す
	if re.search('N', loc):
		lat += lat_width

	# 東端の座標なら、一区画の幅（経度幅）を足す
	if re.search('E', loc):
		long  += long_width

	return (lat, long)     # タプルを返す。

In [11]:
def calc_mesh3 (dlat, dlng):
    mc_1_la_p = dlat * 60 // 40
    mc_1_ln_u = (dlng - 100) // 1
    mc_2_la_q = dlat * 60 % 40 // 5
    mc_2_ln_v = ((dlng - 100) % 1) * 60 // 7.5
    mc_3_la_r = dlat * 60 % 40 % 5 * 60 // 30
    mc_3_ln_w = ((dlng - 100) % 1) * 60 % 7.5 * 60 // 45
    mc = int(mc_1_la_p * 1000000 + mc_1_ln_u * 10000 + mc_2_la_q * 1000 + mc_2_ln_v * 100 + mc_3_la_r * 10 + mc_3_ln_w)
    
    return mc

In [12]:
def add_mesh_row(df):
    mesh_list = []
    for i in df.index:
        lat = df.at[i, 'lat']
        lng = df.at[i, 'lng']
        mesh = calc_mesh3(lat, lng)
        mesh_list.append(mesh)
    df['mesh'] = mesh_list
    return df

In [13]:
df4 = add_mesh_row(df3)
# df3['mesh'].astype('int')

### 1/2地域メッシュコードの作成

In [14]:
def calc_mesh_half (mlat, lat, mlng, lng):
    p_lat = 0
    p_lng = 0

    p_lat = 15 / 3600
    p_lng = 22.5 / 3600
    
    cond_lat_u = mlat + p_lat <= lat < mlat + 2 * p_lat
    cond_lat_d = mlat <= lat < mlat + p_lat
    cond_lng_l = mlng <= lng < mlng + p_lng
    cond_lng_r = mlng + p_lng <= lng < mlng + 2 * p_lng
    
    if cond_lat_d and cond_lng_l:
        return 1
        print(1)
    elif cond_lat_d and cond_lng_r:
        return 2
        print(2)
    elif cond_lat_u and cond_lng_l:
        return 3
        print(3)
    elif cond_lat_u and cond_lng_r:
        return 4
        print(4)
    else:
        return 0
        print(0)

In [15]:
# 3時地域メッシュコードのcolumnsがある場合にそれと使って1/2地域メッシュのコードを作成する
def input_12mcode_to_df(df):
#     print(df.head(10))
    marray = []
        
    for index, row in df.iterrows():
        mlat, mlng = meshcode_to_latlng(row['mesh'], "SW")
        code = calc_mesh_half(mlat, row["lat"], mlng, row["lng"])
        result = row['mesh'] * 10 + code
        marray.append(result)
        
#     df_m12 = pd.DataFrame({'mesh12':marray})
#     print(marray)
#     print(df)
    df["mesh12"] = marray #df_m12
    return df

In [16]:
print(len(df4))
df5 = input_12mcode_to_df(df4)
print(len(df5))

1533
1533


In [17]:
with open('pkls/zansa_12.pkl', 'wb') as f:
    _pickle.dump(df4, f)

In [52]:
mesh12_list = set(list(df4['mesh12']))
mesh_list = list(set(fm_list + list(mesh12_list)))
print(len(mesh_list))

355


In [53]:
with open('pkls/mesh12_list.pkl', 'wb') as f:
    _pickle.dump(mesh12_list, f)
with open('pkls/mesh_list.pkl', 'wb') as f:
    _pickle.dump(mesh_list, f)

### timestampに関して

In [20]:
def mk_ts_list():
    ts_list = []
    for i in df4.index:
        ts_list.append(pd.to_datetime(df4.at[i, 'ts']))
    return ts_list

In [21]:
ts_list = mk_ts_list()
print(len(ts_list))

1533


In [22]:
tsdiv_list = []
a = pd.to_datetime('20161005')
tsdiv_list.append(a)
a = a + datetime.timedelta(days=5)
while a < pd.to_datetime('20171208') :
    tsdiv_list.append(a)
    a = a + datetime.timedelta(days=7)
print(len(tsdiv_list))

62


In [23]:
a = pd.to_datetime('20161020')
b = pd.to_datetime('20161030')
mesh = 53390377
df3[(df3.ts > a) & (df3.ts < b) & (df3.mesh == mesh) & (df.text.str.contains('バ'))]

/Users/tomo/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,ts,lat,lng,text,mesh,mesh12


In [24]:
with open('pkls/tsdiv_list.pkl', 'wb') as f:
    _pickle.dump(tsdiv_list, f)

### 種類の探索方法に関して

In [25]:
def count_tensor(df2):
    index_list = []
    # 布団
    a = list(df2[(df2['text'].str.contains('布団|ふとん|ベッド|ベット|マットレス')) & ~(df2['text'].str.contains('座布団'))].index)
    # じゅうたん類
    b = list(df2[(df2['text'].str.contains('カーペット|じゅうたん|ジュータン|じゅーたん|絨毯|マット|敷物|ゴザ|ござ|ソファ|敷')) & ~(df2['text'].str.contains('マットレス'))].index)
    # 鞄類
    c = list(df2[df2['text'].str.contains('かばん|鞄|バッグ|リュック|カバン|バック')].index)
    # 座布団
    d = list(df2[df2['text'].str.contains('座布団')].index)
    # 枕
    e = list(df2[df2['text'].str.contains('枕|まくら|マクラ')].index)
    # クッション
    f = list(df2[df2['text'].str.contains('クッション|くっしょん|クション|くしょん')].index)
    # 靴
    g = list(df2[df2['text'].str.contains('靴|くつ|クツ|サンダル|スリッパ')].index)
    # イス
    h = list(df2[df2['text'].str.contains('椅子|イス|いす|座椅子|座いす')].index)
    # ダンボール
    i = list(df2[df2['text'].str.contains('ダンボール|段ボール|だんぼーる')].index)
    # 傘
    j = list(df2[df2['text'].str.contains('傘|かさ|カサ')].index)
    # プラスチック類
    k = list(df2[df2['text'].str.contains('プラ')].index)
    # 家電製品
    l = list(df2[df2['text'].str.contains('ストー|機|レンジ|レコーダー|デンシ|パソコン|コンロ|コンポ|ラジカセ|ポット|テレビ|アイロン|ビデオ')].index)
    # 発泡スチロール
    m = list(df2[df2['text'].str.contains('発砲|発泡|ハッポ')].index)
    # おむつ
    n = list(df2[df2['text'].str.contains('オムツ|おむつ')].index)
    # ビン
    o = list(df2[df2['text'].str.contains('ビン|瓶|びん')].index)
    # 雑誌類
    p = list(df2[df2['text'].str.contains('雑誌|紙|雑かみ')].index)
    # 電球・蛍光灯
    q = list(df2[df2['text'].str.contains('光|電球|ライト')].index)
    # 缶・ペット
    r = list(df2[df2['text'].str.contains('缶|かん|カン|ペット')].index)
    # 電池
    s = list(df2[df2['text'].str.contains('電池')].index)
    # ぬいぐるみ・おもちゃ
    t = list(df2[df2['text'].str.contains('ぬいぐるみ|おもちゃ|オモチャ')].index)
    # ガラス 
    u = list(df2[df2['text'].str.contains('ガラス|がらす')].index)
    # ウェットスーツ・サーフボード類
    v = list(df2[df2['text'].str.contains('ウェットスーツ|ボード')].index)
    # ハンガー
    w = list(df2[df2['text'].str.contains('ハンガー|はんがー')].index)
    # その他可燃
    z5 = list(df2[df2['text'].str.contains('可燃|木')].index)
    # その他大型ごみ
    z6 = list(df2[df2['text'].str.contains('大型')].index)
    # その他不燃
    z1 = list(df2[df2['text'].str.contains('不燃')].index)
    # その他資源
    z2 = list(df2[df2['text'].str.contains('資源')].index)
    # その他不明
#     z4 = list(df2[df2['text'].str.contains('nan')].index)
    
    rm_list = a+b+c+d+e+f+g+h+i+j+k+l+m+n+o+p+q+r+s+t+u+v+w+z1+z2+z5+z6
    # その他
    zz_df = df2.drop(rm_list)
    
    
    
    print('a:',len(a),len(b),len(c),len(d),len(e),len(f),len(g),'h:',len(h),len(i),len(j),len(k),len(l),len(m),len(n),'o:',len(o),len(p),len(q),len(r),len(s),len(t),len(u),'v:',len(v),len(w),'z1:',len(z1),len(z2),len(z5),len(z6),len(zz_df.index)),

    return zz_df

In [26]:
condlist = ['布団|ふとん|ベッド|ベット|マットレス',
            'カーペット|じゅうたん|ジュータン|じゅーたん|絨毯|マット|敷物|ゴザ|ござ|ソファ|敷', 
            'かばん|鞄|バッグ|リュック|カバン|バック', 
            '座布団', 
            '枕|まくら|マクラ', 
            'クッション|くっしょん|クション|くしょん', 
            '靴|くつ|クツ|サンダル|スリッパ', 
            '椅子|イス|いす|座椅子|座いす', 
            'ダンボール|段ボール|だんぼーる',
            '傘|かさ|カサ',
            'プラ',
            'ストー|機|レンジ|レコーダー|デンシ|パソコン|コンロ|コンポ|ラジカセ|ポット|テレビ|アイロン|ビデオ',
            '発砲|発泡|ハッポ',
            'オムツ|おむつ',
            'ビン|瓶|びん',
            '雑誌|紙|雑かみ',
            '光|電球|ライト',
            '缶|かん|カン|ペット',
            '電池',
            'ぬいぐるみ|おもちゃ|オモチャ',
            'ガラス|がらす',
            'ウェットスーツ|ボード',
            'ハンガー|はんがー',
            '可燃|木',
            '大型',
            '不燃',
            '資源'
           ]
ncondlist = ['座布団', 'マットレス', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

with open('pkls/condition_list.pkl', 'wb') as f:
    _pickle.dump(condlist, f)
with open('pkls/not_condition_list.pkl', 'wb') as f:
    _pickle.dump(ncondlist, f)

In [27]:
# 種類のリストの作成
kind_list = ['布団類', '絨毯類', '鞄類', '座布団', '枕', 'クッション', '靴', '椅子', 'ダンボール', '傘', 'プラスチック類', '家電製品', '発泡スチロール', 'おむつ', 'ビン', '雑誌類', '電球・蛍光灯', '缶・ペット', '電池', 'ぬいぐるみ・おもちゃ', 'ガラス', 'ウェットスーツ・サーフボード', 'ハンガー', 'その他可燃', 'その他不燃', 'その他資源', 'その他大型', 'その他']
with open('pkls/kind_list.pkl', 'wb') as f:
    _pickle.dump(kind_list, f)

In [45]:
zz_df = count_tensor(df5)
with open('pkls/other_index.pkl', 'wb') as f:
    _pickle.dump(list(zz_df.index), f)

a: 318 483 73 95 151 195 69 h: 16 14 12 39 39 51 4 o: 5 52 10 225 2 11 3 v: 11 6 z1: 167 6 22 69 97


In [29]:
zz_df.index

Int64Index([  10,   14,   16,   24,   26,   31,   47,   55,   66,   75,   76,
              77,   79,  101,  115,  156,  163,  168,  180,  191,  203,  213,
             232,  234,  277,  278,  300,  313,  326,  329,  331,  334,  346,
             355,  367,  379,  381,  385,  405,  411,  422,  436,  441,  502,
             504,  512,  530,  573,  582,  585,  618,  648,  652,  665,  668,
             669,  671,  705,  747,  758,  759,  768,  775,  784,  785,  786,
             787,  790,  811,  816,  821,  823,  832,  833,  852,  866,  896,
             897,  900,  927,  931,  960, 1031, 1095, 1136, 1211, 1228, 1235,
            1271, 1276, 1308, 1317, 1341, 1370, 1454, 1477, 1519],
           dtype='int64')

In [44]:
df5.loc[list(zz_df.index)]

,ts,lat,lng,text,mesh,mesh12
10,2016-10-05 17:08:58,35.3938,139.449,nan,53390375,533903752
14,2016-10-12 16:33:18,35.3955,139.45,nan,53390375,533903752
16,2016-10-12 16:39:52,35.3953,139.45,120番エコリ,53390375,533903752
24,2016-10-14 15:14:15,35.3849,139.454,大鋸3-2-4後出し常習です。お留守でビラを入れても後出しをします。対応出来ますでしょうか。,53390366,533903661
26,2016-10-17 09:04:52,35.3373,139.461,8-301番大きな何かのカバーです。とても硬いです。,53390306,533903062
31,2016-10-20 11:02:23,35.3917,139.458,10-221 ほぼ毎回出ます。 回収済みです。,53390376,533903762
47,2016-10-26 09:50:33,35.3298,139.485,6-332 時計,52397398,523973984
55,2016-10-27 13:07:08,35.3202,139.456,7-352 10月5日で貼られていました。 物干し竿,52397386,523973861
66,2016-10-31 09:16:26,35.3388,139.45,8-286 10月26日でありました。,53390306,533903063
75,2016-11-02 09:06:12,35.34,139.486,5-138未分別,53390308,533903084


In [43]:
df5

,ts,lat,lng,text,mesh,mesh12
3,2016-10-05 10:06:31,35.3926,139.467,2-486 残渣 カーペット,53390377,533903771
4,2016-10-05 10:08:34,35.3889,139.467,2-449 マット類,53390367,533903673
5,2016-10-05 10:10:02,35.3925,139.467,2-486 カーペット,53390377,533903771
6,2016-10-05 10:11:19,35.3908,139.463,2-463 マット類,53390367,533903673
7,2016-10-05 10:13:17,35.3904,139.464,2-467 クッション,53390367,533903673
8,2016-10-05 10:14:36,35.394,139.468,2-177 カーペット,53390377,533903771
9,2016-10-05 10:17:57,35.3927,139.467,2-743 マット類,53390377,533903771
10,2016-10-05 17:08:58,35.3938,139.449,nan,53390375,533903752
11,2016-10-06 10:12:26,35.3952,139.45,藤沢市本町1-8-6 布団 枕,53390375,533903752
14,2016-10-12 16:33:18,35.3955,139.45,nan,53390375,533903752


In [32]:
# 時間軸(62week)，地域メッシュ(352regions)，残渣種類(28kinds)のPanelを作成
# tsdiv_list : 時間軸のリスト
# mesh_list : 地域メッシュのリスト
# kind_list : 種類の名前のリスト

In [33]:
def mk_panel(input_df):
    df = input_df
    for i in range(len(div_list)):
        a = tsdiv_list[i]
        b = tsdiv_list[i+1]
        fname = 'dfts' + str(i)
        fname = df[(df.ts > a) & (df.ts < b)]
    return dfts1

In [34]:
# 1週間ごとのデータに分割し，それぞれをdataframeにして，panelにする．

In [35]:
# 地域メッシュごとのデータに分割し，

In [36]:
# 1週間ごと＆地域メッシュごとのデータから種類をカウントして，dataframeの

In [40]:
nar = np.array([[[1,2,3],[2,3,4]],
                [[3,4,5],[4,5,6]],
                [[5,6,7],[6,7,8]],
                [[7,8,9],[8,9,0]]]
                )

In [41]:
nar.shape

(4, 2, 3)

In [42]:
a = pd.to_datetime('20161004')
b = pd.to_datetime('20171012')
df2[df2.ts < a]

,ts,lat,lng,text
